<a href="https://colab.research.google.com/drive/1trBvrdgyI-Ntd45ZnlT5lhGsI_HnKjC1?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialization  
You can access [this](http://blog.nonewiki.top) get some help

In [ ]:
# Are you using Colab
useColab = True #@param {type:"boolean"}
if useColab:
  # Mount Google Cloud Drive
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive
else:
  %cd .

In [ ]:
# Clone Repo
import os
if not os.path.exists("fish-speech"):
  !git clone https://github.com/fishaudio/fish-speech.git
  !git checkout tags/v1.4.3 # Because the current 1.4 version is more stable, we are using 1.4.3 here. You can replace here
else:
  print("The Fish Speech project is already in the directory.")
# Enter the project directory
%cd fish-speech

In [ ]:
# Install Packages
!sudo apt install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
!pip install pyaudio
!pip install huggingface_hub
!pip install triton
!pip install .
# !huggingface-cli login # If you want to use version 1.5 of the model, please uncomment it
model_id = "fishaudio/fish-speech-1.4" #@param {type:"string"}
download_dir = "checkpoints/fish-speech-1.4" #@param {type:"string"}
!huggingface-cli download {model_id} --local-dir {download_dir} #  You can replace here

Run the necessary variables.

In [ ]:
# Value
vqgan_model = "checkpoints/fish-speech-1.4/firefly-gan-vq-fsq-8x1024-21hz-generator.pth" #@param {type: "string"}
vqgan_config_name = "firefly_gan_vq" #@param {type: "string"}
llama_model = "checkpoints/fish-speech-1.4" #@param {type: "string"}

Fine-tuning

In [ ]:
# Datasets

# Batch extraction of semantic tokens
!python tools/vqgan/extract_vq.py data --num-workers 1 --batch-size 16 --config-name {vqgan_config_name} --checkpoint-path {vqgan_model}

# Pack the dataset into protobuf
input_dir = "data" #@param {type: "string"}
output_dir = "data/protos" #@param {type: "string"}
!python tools/llama/build_dataset.py --input {input_dir} --output {output_dir} --text-extension .lab --num-workers 16

In [ ]:
# Set up the training configuration file

import yaml
project = "Speaker1" #@param {type: "string"}
train_config_name = "text2semantic_finetune" #@param {type: "string"}
lora_config_name = "r_8_alpha_16" #@param {type: "string"}
train_config_path = f"fish_speech/configs/{train_config_name}.yaml"
if not os.path.exists(train_config_path):
  print(f"The file {train_config_path} does not exist.")
  raise FileNotFoundError(train_config_path)
with open(train_config_path, "r", encoding="utf-8") as file:
  data = yaml.load(file, Loader=yaml.FullLoader)
pretrained_ckpt_path = "checkpoints/fish-speech-1.4" #@param {type: "string"}
protos_dir = "data/protos" #@param {type: "string"}
max_steps = 1000 #@param {type: "integer"}
num_workers = 1 #@param {type: "integer"}
batch_size = 1 #@param {type: "integer"}
data["project"] = project
data["pretrained_ckpt_path"] = pretrained_ckpt_path
data["trainer"]["max_steps"] = max_steps
data["data"]["num_workers"] = num_workers
data["data"]["batch_size"] = batch_size
data["train_dataset"]["proto_files"] = [protos_dir]
with open(train_config_path, "w", encoding="utf-8") as file:
  yaml.dump(data, file, allow_unicode=True)

In [ ]:
# Fine-tuning with LoRA
!python fish_speech/train.py --config-name {train_config_name}

In [ ]:
# Convert the LoRA weights to regular weights
lora_weight = "results/Speaker1/checkpoints/step_000000010.ckpt" #@param {type: "string"}
output_path = "checkpoints/fish-speech-1.4-Speaker1-lora/" #@param {type: "string"}
!python tools/llama/merge_lora.py --lora-config {lora_config_name} --base-weight {llama_model} --lora-weight {lora_weight} --output {output}

Inference

In [ ]:
# Generate prompt from voice
prompt_wav = "Speaker1.wav" #@param {type: "string"}
output_npy = "fake.npy" #@param {type: "string"}
!python tools/vqgan/inference.py -i {prompt_wav} -o {output_npy} --checkpoint-path {vqgan_model} --device {device}

In [ ]:
# Generate semantic tokens from text
output_wav_text = "Hello everyone! Welcome to Apple Park" #@param {type: "string"}
output_wav = "fake_Speaker1.wav" #@param {type: "string"}
prompt_wav_text = "Prompt wav text" #@param {type: "string"}
prompt_npy = "fake.npy" #@param {type: "string"}
!python tools/llama/generate.py --text {output_wav_text} --prompt-text {prompt_wav_text} --prompt-tokens {prompt_npy} --checkpoint-path {llama_model} --num-samples 2

# Generate semantic tokens from text
input_npy = "codes_0.npy" #@param {type: "string"}
!python tools/vqgan/inference.py -i {input_npy} -o {output_wav} --device {device} --checkpoint-path {vqgan_model}

# Play
from IPython.display import Audio, display
display(Audio(output_wav, autoplay=False))

Webui

In [ ]:
# Download Cloudflared
!wget https://github.com/cloudflare/cloudflared/releases/download/2024.11.1/cloudflared-linux-386 -O cloudflared
!chmod +x cloudflared

In [ ]:
# Run webui
!./cloudflared tunnel --url 127.0.0.1:7860 | python -m tools.webui --llama-checkpoint-path {llama_model} --decoder-checkpoint-path {vqgan_config_name} --decoder-config-name {vqgan_model}